In [13]:
from keras.models import Sequential, Model
import keras.layers
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, Input
from keras.regularizers import l2
from keras.optimizers import Adam, Adadelta, RMSprop
import keras.losses as losses
from keras.models import load_model
from keras.backend import set_image_data_format
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
import os
import random
import glob
from PIL import Image, ImageOps

set_image_data_format('channels_first')

STILL_ALIVE_REWARD = 1
DEAD_REWARD = -1000

CROP_SHAPE = (750, 539, 1)
RESIZE_WIDTH = 180
RESIZE_HEIGHT = 137
READ_BATCH = 2

MOVES = 5
NONE = 0
RIGHT = 1
DOWN = 2
LEFT = 3
UP = 4

SHOTS_FOLDER = 'data/shots/'

#### Input functions

In [11]:
def readData():
    files = os.listdir(SHOTS_FOLDER)
    
    labels = np.zeros(len(files))    
    data = []
    
    for i, f in enumerate(files):
        im = Image.open(SHOTS_FOLDER + f)
        
        if f in history:
            labels[i] = history[f][2]
        else:
            labels[i] = STILL_ALIVE_REWARD
            
        data.append([np.asarray(im.convert("L"))])
        
    data = np.asarray(data)
    return data, labels


def sampleData(nSamples, propD, propR, propA):
    '''Sample the input data according to some proportions.
    
    Parameters
    ----------
    nSamples: total number of samples
    propD: proportion of samples which represent a death
    propR: proportion of samples which represent a reward
    propA: proportion of rest of samples
    '''
    
    files = os.listdir(SHOTS_FOLDER)
    
    perc = []
    nd, na, nr = (0, 0, 0)
    for f in files:
        r = parseName(f)[3]
        if r == DEAD_REWARD:
            perc.append('d')
            nd += 1
        elif r == STILL_ALIVE_REWARD:
            perc.append('a')
            na += 1
        else:
            perc.append('r')
            nr += 1
        
    n = propD * nd + propA * na + propR * nr
    pd = propD / n
    pa = propA / n
    pr = propR / n
    for i in range(len(perc)):
        if perc[i] == 'd':
            perc[i] = pd
        elif perc[i] == 'a':
            perc[i] = pa
        else:
            perc[i] = pr
        
    samples = np.random.choice(files, size=nSamples, replace=False, p=perc)
    print('Sampled data.')
    return samples


def readSamplesBasic(samples):
    '''Read samples for basic model, which just classifies if a board is finished or not.'''
    data = []
    labels = []
    
    for file in samples:
        im = Image.open(SHOTS_FOLDER + file)
        g, i, m, r = parseName(file)
        
        data.append([np.asarray(im.convert("L"))])
        labels.append(1 if r > 0 else 0)
        
    data = np.asarray(data)
    return data, labels


def getPreviousSample(file):
    '''Get the name of the previous sample.'''
    
    g, i, m, r = parseName(file)
    i -= 1
    
    if i > 0:        
        prevFile = glob.glob(SHOTS_FOLDER + str(g).zfill(3) + '_' + str(i).zfill(3) + '*')[0]
        prevFile = prevFile.replace('\\', '/')
        prevFile = prevFile.split('/')[-1]
        return prevFile      
    else:
        return -1


def getNextSample(file):
    '''Get the name of the next sample.'''
    
    g, i, m, r = parseName(file)
    i += 1
    
    prevFile = glob.glob(SHOTS_FOLDER + str(g).zfill(3) + '_' + str(i).zfill(3) + '*')
    if len(prevFile) == 1:
        prevFile = prevFile[0]
        prevFile = prevFile.replace('\\', '/')
        prevFile = prevFile.split('/')[-1]
        return prevFile
    else:
        return -1
    
    
def readSample(file):
    '''Read a single sample as an array.'''
    im = Image.open(SHOTS_FOLDER + file)
    g, i, m, r = parseName(file)

    data = np.asarray(im.convert("L"))
    label = 1 if r > 0 else 0
    
    return data, label


def parseName(fileName):
    '''Parse the board data from the file name.'''
    return [int(d) for d in fileName[:-4].split('_')]
    
    
def readSampleMove(file):
    '''Read a single sample as an array, also returns the move.'''
    im = Image.open(SHOTS_FOLDER + file)
    g, i, m, r = parseName(file)

    data = np.asarray(im.convert("L"))
    if r == -1000:
        label = -1
    elif r > 1:
        label = r / 100
    else:
        label = 0
#     label = r
    
    return data, m, label


def readBatchSamples(samples, batch):
    '''Read samples in batches, so that we also have information about the movement.'''
    data = []
    labels = []
    
    for file in samples:        
        dataBatch = []
        
        d, label = readSample(file)
        dataBatch.append(d)
        
        # Get previous files
        prevFile = file
        for p in range(batch - 1):
            prevFile = getPreviousSample(prevFile)
            
            if prevFile == -1:
                break
                
            d, _ = readSample(prevFile)
            dataBatch.append(d)
        
        if prevFile != -1:
            dataBatch.reverse()
            data.append(dataBatch)
            labels.append(label)
        
    data = np.asarray(data)
    return data, labels   


def getStateBatches(file, batch):
    '''Get the image batches for the current and next state.'''
          
    s, _, _ = readSampleMove(file)
    
    # Get the image batch for the current state
    curBatch = []
    curBatch.append(s)
    
    # Get previous files
    prevFile = file
    for p in range(batch - 1):
        prevFile = getPreviousSample(prevFile)
        
        if prevFile == -1:
            break
            
        s, _ = readSample(prevFile)
        curBatch.append(s)
        
    if prevFile == -1:
        return -1, -1
    else:
        #Get the image batch for the next state
        nextBatch = []
        
        nextFile = getNextSample(file)
        if nextFile == -1:
            curBatch.reverse()
            return curBatch, curBatch
        
        nextState, _ = readSample(nextFile)        
        
        nextBatch.append(nextState)        
        nextBatch.extend(curBatch[:-1])
        
        curBatch.reverse()
        nextBatch.reverse()
        return curBatch, nextBatch


def readBatchSamplesMove(samples, batch):
    '''Read samples in batches, so that we also have information about the movement.
    Also reads the moves.'''
    states = []
    actions = []
    labels = []
    nextStates = []
    
    for i, file in enumerate(samples):          
        _, a, r = readSampleMove(file)
        
        curBatch, nextBatch = getStateBatches(file, batch)
        if curBatch != -1:
            states.append(curBatch)
            actions.append(a)
            labels.append(r)
            nextStates.append(nextBatch)

        if i % 100 == 0:
            print(i)
        
    states = np.asarray(states)
    actions = np.asarray(actions)
    nextStates = np.asarray(nextStates)
    labels = np.asarray(labels)
       
    # Scale labels
#     labels = np.asarray(labels).reshape(-1, 1)
#     scaler = StandardScaler()
#     scaler.fit(labels)
#     labels = scaler.transform(labels)
    
    return states, actions, labels, nextStates    
    
    
def getModelMoveInput(model, screens):
    '''Get the move from the model which takes moves as input.'''
    
    m = 0
    bestMove = 0
    bestScore = DEAD_REWARD - 1
    
    while m < MOVES:
        score = model.predict([np.asarray([screens]), np.asarray([m])])      
        
        print(m, score)
        if score > bestScore:
            bestScore = score
            bestMove = m
            
        m += 1 

#### DNN basic

Sequential, citeste doar o imagine, nu considera actiunea.

In [9]:
def nn_model_basic():
    '''NN model for basic test.'''
    model = Sequential()
    
    model.add(Conv2D(16, (4, 4), input_shape=(1, RESIZE_HEIGHT, RESIZE_WIDTH), activation='relu', name='conv1'))
    model.add(Conv2D(16, (4, 4), activation='relu', name='conv2'))
    model.add(MaxPool2D(pool_size=(2,2), name='pool1'))
#     model.add(Dropout(0.2))
    
    model.add(Conv2D(8, (4, 4), activation='relu', name='conv3'))
    model.add(Conv2D(8, (4, 4), activation='relu', name='conv4'))
    model.add(MaxPool2D(pool_size=(2,2), name='pool2'))
#     model.add(Dropout(0.2))
    
    model.add(Conv2D(16, (4, 4), activation='relu', name='conv5'))
    model.add(MaxPool2D(pool_size=(2,2), name='pool3'))
#     model.add(Dropout(0.2))
    
    model.add(Flatten(name='flat'))
    model.add(Dense(100, activation='relu', name='dense1'))
    model.add(Dropout(0.5))
    model.add(Dense(40, activation='relu', name='dense2'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid', name='output'))
    
    model.compile(loss='binary_crossentropy',
                 optimizer = 'rmsprop',
                 metrics=['accuracy'])
    
    return model


In [14]:
model = nn_model_basic()
data, labels = readSamplesBasic(sampleData(200, 10, 0.5, 0.5))
print(data.shape)
print(labels)

model.fit(data, labels, epochs=7, batch_size=10, shuffle=True)

Sampled data.
(200, 1, 137, 180)
[1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
Epoch 1/7
 20/200 [==>...........................] - ETA: 50s - loss: 6.7457 - acc: 0.3500 

KeyboardInterrupt: 

#### DNN seq

Sequential, citeste batch-uri de imagini, dar nu considera si actiunea.

In [15]:
def nn_model_seq():
    '''Sequential NN model.'''
    model = Sequential()
    
    model.add(Conv2D(16, (4, 4), input_shape=(READ_BATCH, RESIZE_HEIGHT, RESIZE_WIDTH), activation='relu', name='conv1'))
    model.add(Conv2D(16, (4, 4), activation='relu', name='conv2'))
    model.add(MaxPool2D(pool_size=(2,2), name='pool1'))
#     model.add(Dropout(0.2))
    
    model.add(Conv2D(8, (4, 4), activation='relu', name='conv3'))
    model.add(Conv2D(8, (4, 4), activation='relu', name='conv4'))
    model.add(MaxPool2D(pool_size=(2,2), name='pool2'))
#     model.add(Dropout(0.2))
    
    model.add(Conv2D(16, (4, 4), activation='relu', name='conv5'))
    model.add(MaxPool2D(pool_size=(2,2), name='pool3'))
#     model.add(Dropout(0.2))
    
    model.add(Flatten(name='flat'))
    model.add(Dense(100, activation='relu', name='dense1'))
    model.add(Dropout(0.5))
    model.add(Dense(40, activation='relu', name='dense2'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid', name='output'))
    
    model.compile(loss='binary_crossentropy',
                 optimizer = 'rmsprop',
                 metrics=['accuracy'])
    
    return model


In [16]:
model = nn_model_seq()
data, labels = readBatchSamples(sampleData(200, 10, 0.5, 0.5), READ_BATCH)
print(data.shape)
print(labels)

model.fit(data, labels, epochs=7, batch_size=10, shuffle=True)

Sampled data.
(189, 2, 137, 180)
[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1]
Epoch 1/7


KeyboardInterrupt: 

#### DNN action input

Functional API, citeste batch-uri de imagini, considera actiunea ca un input. 

In [17]:

def nn_model_actionlast():
    '''Full NN model, using the Keras Functional API.
    Action as input, added before the final layer.'''
    screen_input = Input(shape=(READ_BATCH, RESIZE_HEIGHT, RESIZE_WIDTH), name='screen_input')
    
    conv = Conv2D(16, (4, 4), activation='relu', name='conv1')(screen_input)
    conv = Conv2D(16, (4, 4), activation='relu', name='conv2')(conv)
    pool = MaxPool2D(pool_size=(2,2), name='pool1')(conv)
    
    conv = Conv2D(8, (4, 4), activation='relu', name='conv3')(pool)
    conv = Conv2D(8, (4, 4), activation='relu', name='conv4')(conv)
    pool = MaxPool2D(pool_size=(2,2), name='pool2')(conv)
    
    conv = Conv2D(16, (4, 4), activation='relu', name='conv5')(pool)
    pool = MaxPool2D(pool_size=(2,2), name='pool3')(conv)
    
    flat = Flatten(name='flat')(pool)
    dense = Dense(100, activation='relu', name='dense1')(flat)
    drop = Dropout(0.5)(dense)
    dense = Dense(40, activation='relu', name='dense2')(drop)
    drop = Dropout(0.5)(dense)
    
    action_input = Input(shape=(1,), name='action_input')
    concat = keras.layers.concatenate([drop, action_input])
    
    output = Dense(1, activation='linear', name='output')(concat)
    
    model = Model(inputs=[screen_input, action_input], outputs=output)
    
    model.compile(loss='mean_squared_error',
                 optimizer = 'adam',
                 metrics=['accuracy'])
    
    return model



def nn_model_actionmid():
    '''Full NN model, using the Keras Functional API.
    Action as input, added after convolutional layers.'''
    screen_input = Input(shape=(READ_BATCH, RESIZE_HEIGHT, RESIZE_WIDTH), name='screen_input')
    
    conv = Conv2D(16, (4, 4), activation='relu', name='conv1')(screen_input)
    conv = Conv2D(16, (4, 4), activation='relu', name='conv2')(conv)
    pool = MaxPool2D(pool_size=(2,2), name='pool1')(conv)
    
    conv = Conv2D(8, (4, 4), activation='relu', name='conv3')(pool)
    conv = Conv2D(8, (4, 4), activation='relu', name='conv4')(conv)
    pool = MaxPool2D(pool_size=(2,2), name='pool2')(conv)
    
    conv = Conv2D(16, (4, 4), activation='relu', name='conv5')(pool)
    pool = MaxPool2D(pool_size=(2,2), name='pool3')(conv)
    
    flat = Flatten(name='flat')(pool)
    
    action_input = Input(shape=(1,), name='action_input')
    concat = keras.layers.concatenate([flat, action_input])
    
    dense = Dense(100, activation='relu', name='dense1')(concat)
    drop = Dropout(0.5)(dense)
    dense = Dense(40, activation='relu', name='dense2')(dense)
    drop = Dropout(0.5)(dense)
        
    output = Dense(1, activation='linear', name='output')(dense)
    
    model = Model(inputs=[screen_input, action_input], outputs=output)
    
    model.compile(loss='mean_squared_error',
                 optimizer = 'adam',
                 metrics=['accuracy'])
    
    return model
    

In [18]:
model = nn_model_actionlast()
states, actions, labels, nextStates = readBatchSamplesMove(sampleData(200, 5, 5, 1), READ_BATCH)
print(data.shape)

# labels = np.asarray(labels).reshape(-1, 1)
# scaler = StandardScaler()
# scaler.fit(labels)
# labels = scaler.transform(labels)
# print(labels)

model.fit([states, actions], labels, epochs=5, batch_size=32, shuffle=True)

Sampled data.
0
100
(189, 2, 137, 180)
Epoch 1/5
 32/185 [====>.........................] - ETA: 37s - loss: 236.8246 - acc: 0.0000e+00

KeyboardInterrupt: 